In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import time

tf.enable_eager_execution()

In [0]:
def get_tokens():
  with open('/content/drive/My Drive/URE/data/input_token_5_wrds.json') as f:
    inp_vocab = json.load(f)
  with open('/content/drive/My Drive/URE/data/output_token_5_wrds.json') as f:
    out_vocab = json.load(f)
  return inp_vocab, out_vocab

def get_convs():
  with open('/content/drive/My Drive/URE/data/clean_conversations_5_wrds.json') as f:
      convs = json.load(f)
      clean_pairs = {}
  for q, a in convs.items():
      q = q.replace('.', '').replace('*', '').replace('\t', '')
      a = a.replace('.', '').replace('*', '').replace('\t', '')
      q = '\t' + q + '\n'
      a = '\t' + a + '\n'
      clean_pairs[q] = a
    
  return clean_pairs

class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()

    self.create_index()

  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))

    self.vocab = sorted(self.vocab)

    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1

    for word, index in self.word2idx.items():
      self.idx2word[index] = word

In [0]:
input_token, output_token = get_tokens()
pairs = get_convs() 
inp_lang = LanguageIndex(en for en, ma in pairs.items())
targ_lang = LanguageIndex(ma for en, ma in pairs.items())
input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs.items()]
target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs.items()]
max_length_inp = max(len(t) for t in input_tensor)
max_length_tar =  max(len(t) for t in target_tensor)
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                            maxlen=max_length_inp,
                                                            padding='post')
    
target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                              maxlen=max_length_tar, 
                                                              padding='post')

In [0]:
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_activation='sigmoid', 
                                        recurrent_initializer='glorot_uniform')
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)        
    return output, state
    
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, 
                                        return_sequences=True, 
                                        return_state=True,
                                        recurrent_activation='sigmoid',  
                                        recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
    self.W1 = tf.keras.layers.Dense(self.dec_units)
    self.W2 = tf.keras.layers.Dense(self.dec_units)
    self.V = tf.keras.layers.Dense(1)
        
  def call(self, x, hidden, enc_output):
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * enc_output
    context_vector = tf.reduce_sum(context_vector, axis=1)
        
    x = self.embedding(x)
        
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
    output, state = self.gru(x)
        
    output = tf.reshape(output, (-1, output.shape[2]))
        
    x = self.fc(output)
        
    return x, state, attention_weights
        
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
EPOCHS = 25

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([output_token['\t']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 5.6428
Epoch 1 Batch 100 Loss 4.0452
Epoch 1 Batch 200 Loss 3.6984
Epoch 1 Batch 300 Loss 3.6515
Epoch 1 Batch 400 Loss 3.4853
Epoch 1 Batch 500 Loss 3.8514
Epoch 1 Batch 600 Loss 3.5209
Epoch 1 Batch 700 Loss 3.4415
Epoch 1 Batch 800 Loss 3.4915
Epoch 1 Batch 900 Loss 3.3732
Epoch 1 Batch 1000 Loss 3.0474
Epoch 1 Batch 1100 Loss 3.4607
Epoch 1 Batch 1200 Loss 3.3142
Epoch 1 Batch 1300 Loss 3.4671
